<a href="https://colab.research.google.com/github/bhstoller/multi-agent-customer-service/blob/main/notebooks/mcp_server_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MCP Server**

## **Server Components**
This notebook contains the complete MCP (Model Context Protocol) server implementation. Specifically, it provides the Flask-based MCP server (via ngrok for public access) for the customer management tools, including:
- **get_customer:** Retrieve customer by ID
- **list_customers:** List customers with optional status filter
- **update_customer:** Update customer information
- **create_ticket:** Create a support ticket
- **get_customer_history:** Get ticket history for a customer

This notebook exactly mirrors the code in `src/config.py` (configuration) and `src/mcp_server.py` (server implementation).

## **Prerequisites**
1. NGrok Authentication Token in Colab Secrets (as 'NGROK_AUTHTOKEN')
2. Support Database (called `support.db`, created from `database_setup.py`)





## **Clone Repository and Setup**

In [1]:
!git clone https://github.com/bhstoller/multi-agent-customer-service.git

Cloning into 'multi-agent-customer-service'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 314 (delta 89), reused 89 (delta 48), pack-reused 165 (from 1)
Receiving objects: 100% (314/314), 161.36 KiB | 5.98 MiB/s, done.
Resolving deltas: 100% (162/162), done.


In [2]:
!pip install -q -r multi-agent-customer-service/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 12.8 MB/s eta 0:00:00


In [3]:
# Add repo to path
import sys
sys.path.insert(0, '/content/multi-agent-customer-service/src')

## **Configure Notebook**

In [4]:
print("Configuring notebook...")
# Import configuration libraries
from google.colab import userdata
from logging import CRITICAL
import os
import logging

# Set the database configuration
DB_PATH = "/content/support.db"
print(f"- Database path loaded: {DB_PATH}")


# Set NGrok Configuration
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
USE_NGROK = bool(NGROK_AUTHTOKEN)
print(f"- NGrok Authentication token loaded")

# Set up logging
logging.basicConfig(level=CRITICAL, force=True)
logging.getLogger('asyncio').setLevel(CRITICAL)
logging.getLogger('aiohttp').setLevel(CRITICAL)
print(f"- Logging level loaded: CRITICAL")

print("Configuration complete")

Configuring notebook...
- Database path loaded: /content/support.db
- NGrok Authentication token loaded
- Logging level loaded: CRITICAL
Configuration complete


## **Import Dependencies**

In [5]:
print("Importing libraries...")
# Import server libraries
from flask import Flask, request, Response, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import uvicorn


# Import SQL libraries
import sqlite3
import json

# Import general python libraries
from typing import Optional, Dict, List, Any, Callable
from termcolor import colored
import threading
import logging
import time
import requests

# Turn off warnings for readability
import warnings
warnings.filterwarnings('ignore')

print("All imports loaded")

Importing libraries...
All imports loaded


## **Database Initialization**

In [6]:
# Initialize SQLite DB
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cursor = conn.cursor()

# Example table (not used by our tools)
cursor.execute("""
CREATE TABLE IF NOT EXISTS demo (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    note TEXT
)
""")

conn.commit()

print("Database initialized")

Database initialized


### Database Helper Functions

In [7]:
def get_db_connection():
    """Create a database connection with row factory for dict-like access."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

def row_to_dict(row: sqlite3.Row) -> Dict[str, Any]:
    """Convert a SQLite row to a dictionary."""
    return {key: row[key] for key in row.keys()}

### Read MCP Tools

In [8]:
def get_customer(customer_id: int) -> Dict[str, Any]:
    """Retrieve a specific customer by ID."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()
        if row:
            return {'success': True, 'customer': row_to_dict(row)}
        else:
            return {'success': False, 'error': f'Customer with ID {customer_id} not found'}
    except Exception as e:
        return {'success': False, 'error': f'Database error: {str(e)}'}


def list_customers(status: Optional[str] = None) -> Dict[str, Any]:
    """List all customers, optionally filtered by status."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        if status:
            if status not in ['active', 'disabled']:
                return {'success': False, 'error': 'Status must be active or disabled'}
            cursor.execute('SELECT * FROM customers WHERE status = ? ORDER BY name', (status,))
        else:
            cursor.execute('SELECT * FROM customers ORDER BY name')
        rows = cursor.fetchall()
        conn.close()
        customers = [row_to_dict(row) for row in rows]
        return {'success': True, 'count': len(customers), 'customers': customers}
    except Exception as e:
        return {'success': False, 'error': f'Database error: {str(e)}'}


def get_customer_history(customer_id: int) -> Dict[str, Any]:
    """Retrieve complete ticket history for a customer."""
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute('SELECT * FROM tickets WHERE customer_id = ? ORDER BY created_at DESC', (customer_id,))
        rows = cur.fetchall()
        conn.close()
        tickets = [row_to_dict(r) for r in rows]
        return {'success': True, 'count': len(tickets), 'history': tickets}
    except Exception as e:
        return {'success': False, 'error': str(e)}

### Update MCP Tools

In [9]:
def update_customer(customer_id: int, name: Optional[str] = None, email: Optional[str] = None, phone: Optional[str] = None) -> Dict[str, Any]:
    """Update customer information."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {'success': False, 'error': f'Customer with ID {customer_id} not found'}
        updates = []
        params = []
        if name is not None:
            updates.append('name = ?')
            params.append(name.strip())
        if email is not None:
            updates.append('email = ?')
            params.append(email)
        if phone is not None:
            updates.append('phone = ?')
            params.append(phone)
        if not updates:
            conn.close()
            return {'success': False, 'error': 'No fields to update'}
        updates.append('updated_at = CURRENT_TIMESTAMP')
        params.append(customer_id)
        update_clause = ', '.join(updates)
        query = f'UPDATE customers SET {update_clause} WHERE id = ?'
        cursor.execute(query, params)
        conn.commit()
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()
        return {'success': True, 'message': f'Customer {customer_id} updated successfully', 'customer': row_to_dict(row)}
    except Exception as e:
        return {'success': False, 'error': f'Database error: {str(e)}'}

### Write MCP Tools

In [10]:
def create_ticket(customer_id: int, issue: str, priority: str) -> Dict[str, Any]:
    """Create a new support ticket for a customer."""
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute('SELECT id FROM customers WHERE id = ?', (customer_id,))
        if not cur.fetchone():
            conn.close()
            return {'success': False, 'error': f'Customer {customer_id} not found'}
        cur.execute('INSERT INTO tickets (customer_id, issue, status, priority, created_at) VALUES (?, ?, ?, ?, CURRENT_TIMESTAMP)', (customer_id, issue, 'open', priority))
        ticket_id = cur.lastrowid
        conn.commit()
        cur.execute('SELECT * FROM tickets WHERE id = ?', (ticket_id,))
        row = cur.fetchone()
        conn.close()
        return {'success': True, 'ticket': row_to_dict(row)}
    except Exception as e:
        return {'success': False, 'error': str(e)}

### MCP TOOL Availability

In [11]:
print("Customer and ticket management functions defined successfully!")
print("Available functions:")
print("   - get_customer(customer_id)")
print("   - list_customers(status=None)")
print("   - update_customer(customer_id, data)")
print("   - create_ticket(customer_id, issue, priority)")
print("   - get_customer_history(customer_id)")

Customer and ticket management functions defined successfully!
Available functions:
   - get_customer(customer_id)
   - list_customers(status=None)
   - update_customer(customer_id, data)
   - create_ticket(customer_id, issue, priority)
   - get_customer_history(customer_id)


In [12]:
print("Fetching Customer ID 1:")
result = get_customer(1)
print(result)
if result['success']:
    customer = result['customer']
    print(f"   Name: {customer['name']}")
    print(f"   Email: {customer['email']}")
    print(f"   Status: {customer['status']}")

Fetching Customer ID 1:
{'success': True, 'customer': {'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com', 'phone': '+1-555-0101', 'status': 'active', 'created_at': '2025-11-17 21:35:02', 'updated_at': '2025-11-17 21:35:02'}}
   Name: John Doe
   Email: john.doe@example.com
   Status: active


## **MCP Tool Registration**

In [13]:
MCP_TOOLS = [
    {
        'name': 'get_customer',
        'description': 'Retrieve a customer record by its ID.',
        'inputSchema': {'type': 'object', 'properties': {'customer_id': {'type': 'integer', 'description': 'The ID of the customer to retrieve.'}}, 'required': ['customer_id']}
    },
    {
        'name': 'list_customers',
        'description': 'List all customers, optionally filtered by status.',
        'inputSchema': {'type': 'object', 'properties': {'status': {'type': 'string', 'enum': ['active', 'disabled'], 'description': 'Filter by customer status.'}}, 'required': []}
    },
    {
        'name': 'update_customer',
        'description': 'Update customer information.',
        'inputSchema': {'type': 'object', 'properties': {'customer_id': {'type': 'integer'}, 'name': {'type': 'string'}, 'email': {'type': 'string'}, 'phone': {'type': 'string'}}, 'required': ['customer_id']}
    },
    {
        'name': 'create_ticket',
        'description': 'Create a new support ticket.',
        'inputSchema': {'type': 'object', 'properties': {'customer_id': {'type': 'integer'}, 'issue': {'type': 'string'}, 'priority': {'type': 'string', 'enum': ['low', 'medium', 'high']}}, 'required': ['customer_id', 'issue', 'priority']}
    },
    {
        'name': 'get_customer_history',
        'description': 'Get all tickets for a customer.',
        'inputSchema': {'type': 'object', 'properties': {'customer_id': {'type': 'integer'}}, 'required': ['customer_id']}
    }
]

In [14]:
def create_sse_message(data: Dict[str, Any]) -> str:
    return f'data: {json.dumps(data)}' + chr(10) + chr(10)


def handle_initialize(message: Dict[str, Any]) -> Dict[str, Any]:
    return {
        'jsonrpc': '2.0',
        'id': message.get('id'),
        'result': {
            'protocolVersion': '2024-11-05',
            'capabilities': {'tools': {}},
            'serverInfo': {'name': 'customer-management-server', 'version': '1.0.0'}
        }
    }


def handle_tools_list(message: Dict[str, Any]) -> Dict[str, Any]:
    return {'jsonrpc': '2.0', 'id': message.get('id'), 'result': {'tools': MCP_TOOLS}}


def handle_tools_call(message: Dict[str, Any]) -> Dict[str, Any]:
    params = message.get('params', {})
    tool_name = params.get('name')
    arguments = params.get('arguments', {})
    tool_functions = {
        'get_customer': get_customer,
        'list_customers': list_customers,
        'update_customer': update_customer,
        'create_ticket': create_ticket,
        'get_customer_history': get_customer_history,
    }
    if tool_name not in tool_functions:
        return {'jsonrpc': '2.0', 'id': message.get('id'), 'error': {'code': -32601, 'message': f'Tool not found: {tool_name}'}}
    try:
        result = tool_functions[tool_name](**arguments)
        return {'jsonrpc': '2.0', 'id': message.get('id'), 'result': {'content': [{'type': 'text', 'text': json.dumps(result, indent=2)}]}}
    except Exception as e:
        return {'jsonrpc': '2.0', 'id': message.get('id'), 'error': {'code': -32603, 'message': f'Tool execution error: {str(e)}'}}


def process_mcp_message(message: Dict[str, Any]) -> Dict[str, Any]:
    method = message.get('method')
    if method == 'initialize':
        return handle_initialize(message)
    if method == 'tools/list':
        return handle_tools_list(message)
    if method == 'tools/call':
        return handle_tools_call(message)
    return {'jsonrpc': '2.0', 'id': message.get('id'), 'error': {'code': -32601, 'message': f'Method not found: {method}'}}

In [15]:
print("MCP Server implementation complete!")
print("Server features:")
print("   - MCP protocol support (2024-11-05)")
print("   - Server-Sent Events (SSE) streaming")
print(f"   - {len(MCP_TOOLS)} tools exposed")
print("   - Health check endpoint")
print("   - CORS enabled for cross-origin requests")

MCP Server implementation complete!
Server features:
   - MCP protocol support (2024-11-05)
   - Server-Sent Events (SSE) streaming
   - 5 tools exposed
   - Health check endpoint
   - CORS enabled for cross-origin requests


## **Flask Application Configuration**

In [16]:
# Create Flask app
app = Flask(__name__)
CORS(app)

# Server state
server_thread: Optional[threading.Thread] = None
server_running: bool = False

# FLASK ENDPOINTS
@app.route("/mcp", methods=["POST"])
def mcp_endpoint() -> Response:
    """
    Main MCP endpoint.
    Receives JSON-RPC messages and streams responses via SSE.

    Returns:
        A Flask Response streaming SSE-formatted MCP output.
    """
    message: Dict[str, Any] = request.get_json()

    def generate():
        try:
            response = process_mcp_message(message)
            yield create_sse_message(response)
        except Exception as e:
            error_response = {
                "jsonrpc": "2.0",
                "id": None,
                "error": {
                    "code": -32700,
                    "message": f"Parse error: {str(e)}"
                }
            }
            yield create_sse_message(error_response)

    return Response(generate(), mimetype="text/event-stream")

@app.route("/health", methods=["GET"])
def health_check() -> Response:
    """
    Health check endpoint.

    Returns:
        A JSON response indicating server health.
    """
    return jsonify({
        "status": "healthy",
        "server": "customer-management-mcp-server",
        "version": "1.0.0"
    })

## **Running Server**

In [17]:
ngrok_auth = userdata.get('NGROK_AUTHTOKEN')
ngrok.kill()

In [18]:
# Server configuration
SERVER_HOST = '0.0.0.0'
SERVER_PORT = 5000
SERVER_URL = f'http://{SERVER_HOST}:{SERVER_PORT}'

def run_server():
    """Run the Flask server in a separate thread."""
    global server_running
    server_running = True
    app.run(host=SERVER_HOST, port=SERVER_PORT, debug=False, use_reloader=False)

def start_server(use_ngrok=True):
    """Start the MCP server in a background thread."""
    global server_thread, server_running

    if server_thread and server_thread.is_alive():
        print(colored("Server is already running!", "yellow"))
        return

    print(colored("Starting MCP server...", "cyan"))

    # Start server in background thread
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()

    # Wait for server to start
    time.sleep(2)

    # Check if server is healthy
    try:
        response = requests.get(f'{SERVER_URL}/health', timeout=5)
        if response.status_code == 200:
            print(colored("MCP Server is running!", "green"))
            print(colored(f"Local URL: {SERVER_URL}", "cyan"))

            # Set up ngrok tunnel if requested
            if use_ngrok:
                print(colored("\nSetting up public tunnel with ngrok...", "cyan"))
                try:
                    # Get ngrok authtoken from Colab secrets
                    try:
                        ngrok.set_auth_token(ngrok_auth)
                        print(colored("Ngrok authenticated", "green"))
                    except Exception as e:
                        print(colored("NGROK_AUTHTOKEN not found in Colab secrets", "yellow"))
                        print(colored("   To use ngrok:", "yellow"))
                        print(colored("   1. Get free authtoken from https://ngrok.com", "yellow"))
                        print(colored("   2. In Colab: Click (Secrets) in left sidebar", "yellow"))
                        print(colored("   3. Add secret: Name='NGROK_AUTHTOKEN', Value=<your-token>", "yellow"))
                        print(colored("   4. Enable 'Notebook access' for the secret", "yellow"))
                        print(colored("   5. Re-run this cell", "yellow"))
                        print(colored("\n   Server is still accessible locally at " + SERVER_URL, "cyan"))
                        return

                    # Create ngrok tunnel
                    public_url = ngrok.connect(SERVER_PORT)
                    print(colored(f"Public URL: {public_url}", "green", attrs=["bold"]))
                    print(colored(f"MCP Endpoint: {public_url}/mcp", "green", attrs=["bold"]))
                    print(colored(f"Health Check: {public_url}/health", "cyan"))
                    print()
                    print(colored("MCP Inspector Instructions:", "yellow", attrs=["bold"]))
                    print(colored("1. Run in terminal: npx @modelcontextprotocol/inspector", "yellow"))
                    print(colored("2. This will open MCP Inspector in your browser", "yellow"))
                    print(colored(f"3. Enter MCP URL: {public_url}/mcp", "yellow"))
                    print(colored("4. Click 'Connect' and test the customer management tools!", "yellow"))
                except Exception as e:
                    if "NGROK_AUTHTOKEN" not in str(e):
                        print(colored(f"Could not set up ngrok tunnel: {e}", "yellow"))
                        print(colored("Server is still accessible locally", "yellow"))
        else:
            print(colored("Server started but health check failed", "red"))
    except Exception as e:
        print(colored(f"Failed to connect to server: {e}", "red"))

def stop_server():
    """Stop the MCP server."""
    global server_running
    server_running = False
    print(colored("Server stopped", "yellow"))
    print(colored("   Note: In Colab, the thread will continue until the runtime is reset", "yellow"))

def check_server_status():
    """Check if the server is running."""
    try:
        response = requests.get(f'{SERVER_URL}/health', timeout=2)
        if response.status_code == 200:
            print(colored("Server is running and healthy", "green"))
            health_data = response.json()
            print(f"   Status: {health_data['status']}")
            print(f"   Server: {health_data['server']}")
            print(f"   Version: {health_data['version']}")
            return True
        else:
            print(colored("Server is not responding correctly", "red"))
            return False
    except Exception as e:
        print(colored("Server is not running", "red"))
        print(f"   Error: {e}")
        return False

# Start the server
start_server()

Starting MCP server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:20:32] "GET /health HTTP/1.1" 200 -


MCP Server is running!
Local URL: http://0.0.0.0:5000

Setting up public tunnel with ngrok...
Ngrok authenticated
Could not set up ngrok tunnel: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The endpoint 'https://polar-nonsolubly-madden.ngrok-free.dev' is already online. Either\n1. stop your existing endpoint first, or\n2. start both endpoints with `--pooling-enabled` to load balance between them.\r\n\r\nERR_NGROK_334\r\n"}}

Server is still accessible locally


## **MCP Tool Testing**

### Connection Test

In [19]:
def send_mcp_message(method: str, params: dict = None, message_id: int = 1):
    """
    Send an MCP message to the server and display the request/response.
    """
    # Construct MCP message
    message = {
        "jsonrpc": "2.0",
        "id": message_id,
        "method": method
    }

    if params:
        message["params"] = params

    print(colored(f"\nSending MCP Request:", "cyan", attrs=["bold"]))
    print(colored(json.dumps(message, indent=2), "cyan"))

    try:
        # Send request to MCP endpoint
        response = requests.post(
            f'{SERVER_URL}/mcp',
            json=message,
            headers={'Content-Type': 'application/json'},
            stream=True,
            timeout=10
        )

        # Parse SSE response
        for line in response.iter_lines():
            if line:
                line_str = line.decode('utf-8')
                if line_str.startswith('data: '):
                    data = json.loads(line_str[6:])  # Remove 'data: ' prefix

                    print(colored(f"\nReceived MCP Response:", "green", attrs=["bold"]))
                    print(colored(json.dumps(data, indent=2), "green"))

                    return data

    except Exception as e:
        print(colored(f"\nError: {e}", "red"))
        return None

# Test 1: Initialize
print(colored("="*60, "magenta"))
print(colored("MCP CONNECTION TEST", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

init_response = send_mcp_message(
    method="initialize",
    params={
        "protocolVersion": "2024-11-05",
        "capabilities": {},
        "clientInfo": {
            "name": "colab-test-client",
            "version": "1.0.0"
        }
    },
    message_id=1
)

if init_response and 'result' in init_response:
    print(colored("\nInitialization successful!", "green", attrs=["bold"]))
    print(f"   Protocol Version: {init_response['result']['protocolVersion']}")
    print(f"   Server: {init_response['result']['serverInfo']['name']}")
else:
    print(colored("\nInitialization failed", "red", attrs=["bold"]))

INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:20:36] "POST /mcp HTTP/1.1" 200 -


MCP CONNECTION TEST

Sending MCP Request:
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "initialize",
  "params": {
    "protocolVersion": "2024-11-05",
    "capabilities": {},
    "clientInfo": {
      "name": "colab-test-client",
      "version": "1.0.0"
    }
  }
}

Received MCP Response:
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "protocolVersion": "2024-11-05",
    "capabilities": {
      "tools": {}
    },
    "serverInfo": {
      "name": "customer-management-server",
      "version": "1.0.0"
    }
  }
}

Initialization successful!
   Protocol Version: 2024-11-05
   Server: customer-management-server


### Tools Availability Test

In [20]:
print(colored("="*60, "magenta"))
print(colored("TOOL AVAILABILITY TEST", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

tools_response = send_mcp_message(
    method="tools/list",
    message_id=2
)

if tools_response and 'result' in tools_response:
    tools = tools_response['result']['tools']
    print(colored(f"\nFound {len(tools)} tools:", "green", attrs=["bold"]))
    for i, tool in enumerate(tools, 1):
        print(colored(f"\n{i}. {tool['name']}", "yellow", attrs=["bold"]))
        print(f"   {tool['description']}")
else:
    print(colored("\nFailed to list tools", "red", attrs=["bold"]))

INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:20:36] "POST /mcp HTTP/1.1" 200 -


TOOL AVAILABILITY TEST

Sending MCP Request:
{
  "jsonrpc": "2.0",
  "id": 2,
  "method": "tools/list"
}

Received MCP Response:
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "tools": [
      {
        "name": "get_customer",
        "description": "Retrieve a customer record by its ID.",
        "inputSchema": {
          "type": "object",
          "properties": {
            "customer_id": {
              "type": "integer",
              "description": "The ID of the customer to retrieve."
            }
          },
          "required": [
            "customer_id"
          ]
        }
      },
      {
        "name": "list_customers",
        "description": "List all customers, optionally filtered by status.",
        "inputSchema": {
          "type": "object",
          "properties": {
            "status": {
              "type": "string",
              "enum": [
                "active",
                "disabled"
              ],
              "description": "Filter by c

### get_customer Tool Test

In [21]:
print(colored("="*60, "magenta"))
print(colored("GET CUSTOMER TOOL TEST", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

get_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "get_customer",
        "arguments": {
            "customer_id": 1
        }
    },
    message_id=4
)

if get_response and 'result' in get_response:
    content = get_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        customer = data['customer']
        print(colored("\nCustomer found:", "green", attrs=["bold"]))
        print(f"   ID: {customer['id']}")
        print(f"   Name: {customer['name']}")
        print(f"   Email: {customer['email']}")
        print(f"   Phone: {customer['phone']}")
        print(f"   Status: {customer['status']}")
        print(f"   Created: {customer['created_at']}")
    else:
        print(colored(f"\nError: {data['error']}", "red"))
else:
    print(colored("\nTool call failed", "red", attrs=["bold"]))

INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:20:36] "POST /mcp HTTP/1.1" 200 -


GET CUSTOMER TOOL TEST

Sending MCP Request:
{
  "jsonrpc": "2.0",
  "id": 4,
  "method": "tools/call",
  "params": {
    "name": "get_customer",
    "arguments": {
      "customer_id": 1
    }
  }
}

Received MCP Response:
{
  "jsonrpc": "2.0",
  "id": 4,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"customer\": {\n    \"id\": 1,\n    \"name\": \"John Doe\",\n    \"email\": \"john.doe@example.com\",\n    \"phone\": \"+1-555-0101\",\n    \"status\": \"active\",\n    \"created_at\": \"2025-11-17 21:35:02\",\n    \"updated_at\": \"2025-11-17 21:35:02\"\n  }\n}"
      }
    ]
  }
}

Customer found:
   ID: 1
   Name: John Doe
   Email: john.doe@example.com
   Phone: +1-555-0101
   Status: active
   Created: 2025-11-17 21:35:02


### list_customer Tool Test

In [22]:
print(colored("="*60, "magenta"))
print(colored("LIST ALL CUSTOMERS TEST", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

list_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "list_customers",
        "arguments": {}
    },
    message_id=3
)

if list_response and 'result' in list_response:
    content = list_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        print(colored(f"\nFound {data['count']} customers:", "green", attrs=["bold"]))
        for customer in data['customers']:
            status_color = "green" if customer['status'] == 'active' else "red"
            print(f"\n   ID: {customer['id']}")
            print(f"   Name: {customer['name']}")
            print(f"   Email: {customer['email']}")
            print(f"   Phone: {customer['phone']}")
            print(colored(f"   Status: {customer['status']}", status_color))
    else:
        print(colored(f"\nError: {data['error']}", "red"))
else:
    print(colored("\nTool call failed", "red", attrs=["bold"]))

INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:20:36] "POST /mcp HTTP/1.1" 200 -


LIST ALL CUSTOMERS TEST

Sending MCP Request:
{
  "jsonrpc": "2.0",
  "id": 3,
  "method": "tools/call",
  "params": {
    "name": "list_customers",
    "arguments": {}
  }
}

Received MCP Response:
{
  "jsonrpc": "2.0",
  "id": 3,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"count\": 30,\n  \"customers\": [\n    {\n      \"id\": 4,\n      \"name\": \"Alice Williams\",\n      \"email\": \"alice.w@techcorp.com\",\n      \"phone\": \"+1-555-0104\",\n      \"status\": \"active\",\n      \"created_at\": \"2025-11-17 21:35:02\",\n      \"updated_at\": \"2025-11-17 21:35:02\"\n    },\n    {\n      \"id\": 19,\n      \"name\": \"Alice Williams\",\n      \"email\": \"alice.w@techcorp.com\",\n      \"phone\": \"+1-555-0104\",\n      \"status\": \"active\",\n      \"created_at\": \"2025-11-20 01:02:56\",\n      \"updated_at\": \"2025-11-20 01:02:56\"\n    },\n    {\n      \"id\": 3,\n      \"name\": \"Bob Johnson\",\n      \"email\"

### update_customer Tool Test

In [23]:
print(colored("="*60, "magenta"))
print(colored("UPDATE CUSTOMER INFORMATION TEST", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

update_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "update_customer",
        "arguments": {
            "customer_id": 2,
            "email": "bob.smith.updated@email.com",
            "phone": "+1-555-9999"
        }
    },
    message_id=6
)

if update_response and 'result' in update_response:
    content = update_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        customer = data['customer']
        print(colored(f"\n{data['message']}", "green", attrs=["bold"]))
        print(f"   ID: {customer['id']}")
        print(f"   Name: {customer['name']}")
        print(colored(f"   Email: {customer['email']} (updated)", "yellow"))
        print(colored(f"   Phone: {customer['phone']} (updated)", "yellow"))
        print(f"   Updated at: {customer['updated_at']}")
    else:
        print(colored(f"\nError: {data['error']}", "red"))
else:
    print(colored("\nTool call failed", "red", attrs=["bold"]))

INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:20:36] "POST /mcp HTTP/1.1" 200 -


UPDATE CUSTOMER INFORMATION TEST

Sending MCP Request:
{
  "jsonrpc": "2.0",
  "id": 6,
  "method": "tools/call",
  "params": {
    "name": "update_customer",
    "arguments": {
      "customer_id": 2,
      "email": "bob.smith.updated@email.com",
      "phone": "+1-555-9999"
    }
  }
}

Received MCP Response:
{
  "jsonrpc": "2.0",
  "id": 6,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"message\": \"Customer 2 updated successfully\",\n  \"customer\": {\n    \"id\": 2,\n    \"name\": \"Jane Smith\",\n    \"email\": \"bob.smith.updated@email.com\",\n    \"phone\": \"+1-555-9999\",\n    \"status\": \"active\",\n    \"created_at\": \"2025-11-17 21:35:02\",\n    \"updated_at\": \"2025-11-20 20:20:36\"\n  }\n}"
      }
    ]
  }
}

Customer 2 updated successfully
   ID: 2
   Name: Jane Smith
   Email: bob.smith.updated@email.com (updated)
   Phone: +1-555-9999 (updated)
   Updated at: 2025-11-20 20:20:36


### create_ticket Tool Test

In [24]:
print(colored("="*60, "magenta"))
print(colored("CREATE TICKET TEST", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

ticket_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "create_ticket",
        "arguments": {
            "customer_id": 1,
            "issue": "User cannot log in after password reset.",
            "priority": "high"
        }
    },
    message_id=6
)

# Parse the response
if ticket_response and "result" in ticket_response:
    try:
        content = ticket_response["result"]["content"][0]["text"]
        data = json.loads(content)

        if data.get("success", False):
            ticket = data["ticket"]

            print(colored(f"\nTicket successfully created!", "green", attrs=["bold"]))
            print(f"   Ticket ID: {ticket['id']}")
            print(f"   Customer ID: {ticket['customer_id']}")
            print(f"   Issue: {ticket['issue']}")
            print(f"   Priority: {ticket['priority']}")
            print(f"   Status: {ticket['status']}")
            print(f"   Created: {ticket['created_at']}")
        else:
            print(colored(f"\nError: {data.get('error', 'Unknown error')}", "red"))

    except Exception as e:
        print(colored(f"\nParsing error: {e}", "red"))

else:
    print(colored("\nTool call failed", "red", attrs=["bold"]))

INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:20:36] "POST /mcp HTTP/1.1" 200 -


CREATE TICKET TEST

Sending MCP Request:
{
  "jsonrpc": "2.0",
  "id": 6,
  "method": "tools/call",
  "params": {
    "name": "create_ticket",
    "arguments": {
      "customer_id": 1,
      "issue": "User cannot log in after password reset.",
      "priority": "high"
    }
  }
}

Received MCP Response:
{
  "jsonrpc": "2.0",
  "id": 6,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"ticket\": {\n    \"id\": 51,\n    \"customer_id\": 1,\n    \"issue\": \"User cannot log in after password reset.\",\n    \"status\": \"open\",\n    \"priority\": \"high\",\n    \"created_at\": \"2025-11-20 20:20:36\"\n  }\n}"
      }
    ]
  }
}

Ticket successfully created!
   Ticket ID: 51
   Customer ID: 1
   Issue: User cannot log in after password reset.
   Priority: high
   Status: open
   Created: 2025-11-20 20:20:36


### get_customer_history Tool Test

In [25]:
print(colored("="*60, "magenta"))
print(colored("GET CUSTOMER HISTORY TEST", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

history_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "get_customer_history",
        "arguments": {
            "customer_id": 1
        }
    },
    message_id=7
)

# Parse the response
if history_response and "result" in history_response:
    try:
        content = history_response["result"]["content"][0]["text"]
        data = json.loads(content)

        if data.get("success", False):
            print(colored(f"\nRetrieved customer history successfully!", "green", attrs=["bold"]))
            print(f"   Total Tickets: {data['count']}")

            if data['count'] > 0:
                print("\nTicket History:")
                for t in data["history"]:
                    print(f"   • Ticket ID: {t['id']}")
                    print(f"     Issue: {t['issue']}")
                    print(f"     Priority: {t['priority']}")
                    print(f"     Status: {t['status']}")
                    print(f"     Created: {t['created_at']}")
                    print()

        else:
            print(colored(f"\nError: {data.get('error', 'Unknown error')}", "red"))

    except Exception as e:
        print(colored(f"\nParsing error: {e}", "red"))

else:
    print(colored("\nTool call failed", "red", attrs=["bold"]))

INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:20:36] "POST /mcp HTTP/1.1" 200 -


GET CUSTOMER HISTORY TEST

Sending MCP Request:
{
  "jsonrpc": "2.0",
  "id": 7,
  "method": "tools/call",
  "params": {
    "name": "get_customer_history",
    "arguments": {
      "customer_id": 1
    }
  }
}

Received MCP Response:
{
  "jsonrpc": "2.0",
  "id": 7,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"count\": 5,\n  \"history\": [\n    {\n      \"id\": 51,\n      \"customer_id\": 1,\n      \"issue\": \"User cannot log in after password reset.\",\n      \"status\": \"open\",\n      \"priority\": \"high\",\n      \"created_at\": \"2025-11-20 20:20:36\"\n    },\n    {\n      \"id\": 26,\n      \"customer_id\": 1,\n      \"issue\": \"Cannot login to account\",\n      \"status\": \"open\",\n      \"priority\": \"high\",\n      \"created_at\": \"2025-11-20 01:02:56\"\n    },\n    {\n      \"id\": 31,\n      \"customer_id\": 1,\n      \"issue\": \"Password reset not working\",\n      \"status\": \"in_progress\",\n      

## **Stop Server**

In [26]:
# Stop the server
stop_server()

Server stopped
   Note: In Colab, the thread will continue until the runtime is reset
